In [1]:
import pandas as pd
import plotly as plt
import matplotlib as mplt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

### State 0

In [2]:
rs = pd.read_csv('result1_rs.csv')
kdb = pd.read_csv('result1_kdb.csv')
table = [rs,kdb]

In [3]:
def add_sum_fc(tab):
    col = tab['force call minimization'] + tab['force call saddle']
    col = pd.DataFrame(col)
    col = col.rename(columns={0:"sum force call"})
    new_tab = pd.concat([tab[['proc #','rate','force call minimization','force call saddle']],col],axis = 1)
    return new_tab

In [4]:
new_table = []
for tab in table:
    new_tab = add_sum_fc(tab)
    new_table.append(new_tab)

In [7]:
def plotFc_vs_Err(table):
    
    totalrate = []
    for tab in table:
        total = sum(list(tab['rate']))
        totalrate.append(total)
    
    error = []
    fc = []
    for i in range(len(table)):
        tab = table[i]
        rErr = []
        s_fc = []
        r = 0
        s = 0
        for j in range(len(tab)):
            r += tab['rate'].loc[j]
            s += tab['sum force call'].loc[j]
            e = abs(r - totalrate[i])/totalrate[i]
            rErr.append(e)
            s_fc.append(s)
        error.append(rErr)
        fc.append(s_fc)

    data = []
    color = ['purple','orange']
    name_list = ['Random Searching', 'KDB']
    for i in range(len(table)):
        data.append(go.Scatter(x = fc[i], y = error[i], marker_color = color[i], name = name_list[i]))
    fig = go.Figure(data)
    fig.update_layout(xaxis_title = '<b> Force calls <b>', yaxis_title = '<b> Relative Error in the Total Rate <b>',
                  legend_title = '<b> Method <b>', yaxis_showgrid=False, xaxis_showgrid=False)
    fig.show()

In [8]:
plotFc_vs_Err(new_table)